# Lernaufgabe 2: Hamming-Kodierung

Sie kennen bereits die Hamming-Kodierung aus dem vorherigen Kapitel. Zur Vertiefung des Gelernten, und um`jupyter` kennenzulernen, implementieren Sie sie in dieser und den folgenden Lernaufgaben mit `python`.

🤔 Doch wie funktioniert `jupyter`? Jede Datei in `jupyter` ist ein Notizbuch, *notebook* zu Englisch, welches aus *Zellen* aufgebaut ist. Jede Zelle kann entweder Text oder `python`-Code enthalten. Wenn Sie eine Code-Zelle anwählen, und die Tasten ⬆️ + ⏎ drücken, wird der Code in dieser Zelle ausgeführt, und das Resultat darunter angezeigt. Das Bemerkenswerte hierbei ist, dass die Berechnungen und deren Resultate aus einer Zelle in einer anderen Zelle weiterverarbeitet werden können.

👏 Ihre Aufgabe ist nun jeweils, die Kommentare im Format `# TODO: ...` durch funktionierenden Code zu ersetzen.

## Teilaufgabe 2.1

Damit wir testen können, ob Ihre Implementierung am Ende auch tatsächlich funktioniert, müssen wir ein *Testgeschirr* haben; in unserem Fall die Simulation eines unsicheren Mediums sowie verschiedene Funktionen zur Kodierung und Dekodierung der Nachrichten. Nachfolgend verwenden wir folgende Terminologie:

* Die Nachricht `message` ist ein Text, der für Menschen lesbar ist
* Diese hat eine Repräsentation als `bits`, also eine Folge von `0` und `1`. In der nachfolgenden Implementation verwenden wir hierzu die Datenstruktur eines `bitarray`s, welche sich sehr ähnlich wie das Ihnen bereits bekannte `array` verhält.
* In Anlehnung an die Erklärung mit dem Kartentrick aus dem Buch können wir diese `bits` auch als `table` darstellen, welches in unserem Fall ein `array` von `bitarray`s ist.

⚠️ Der Fokus in diesem Kapitel liegt auf einem vertieften Verständnis der selbstkorrigierenden Kodierungen. Wir lassen darum das Problem der Suche nach einer optimalen Dartstellungsform der Nachricht außen vor, und nehmen für unsere Kartentrick-Tabelle immer eine fixe Anzahl von Spalten, nämlich `4`.

In groben Zügen läuft der Test, ob die Hamming-Kodierung richtig implementiert wurde, wie folgt ab:

1. Die natürlichsprachliche Nachricht `message` wird zu `bits` und zum `table` konvertiert
2. Daraufhin wird sie kodiert (`encoded_...`)
3. Für die Übertragung wird sie durch ein unzuverlässiges Medium geschickt, in welchem einige Bits verändert werden können (`corrupted_...`)
4. Danach wird versucht, den Fehler zu korrigieren (`corrected_...`)
5. Die `corrected_bits` und `corrected_table` werden zur `decoded_message` konvertiert

In [1]:
from bitarray import bitarray

def message_to_bits(message):
    """
    Konvertiert einen gegebenen string in seine Repräsentation als Reihe 
    von bits in Form eines bitarray.
    """
    bits = bitarray()
    bits.frombytes(message.encode("ascii"))
    return bits

def bits_to_message(bits):
    """
    Versucht eine Reihe von bits in Form eines bitarray in einen string zu konvertieren.
    bit-Folgen, die keinem Symbol zugeordnet werden können, werden in ein � umgewandelt.
    """
    bees = bits.tobytes()
    message = ""
    for bee in bees:
        try:
            message += bytes([bee]).decode("ascii")
        except:
            message += "�"
    return message

assert message_to_bits("Hello") == bitarray('0100100001100101011011000110110001101111')
assert bits_to_message(bitarray('0100100001100101011011000110110001101111')) == "Hello"

In der obigen Zelle erhalten Sie zwei Helfer-Funktionen, welche Ihnen den Umgang mit bits erleichtern. Wir verwenden zur Darstellung von bit-Folgen die Bibliothek [`bitarray`](https://pypi.org/project/bitarray/), welche uns dem Umgang damit erleichtert. Den genauen Umgang damit müssen Sie nicht verstehen, es reicht aus zu wissen, dass Sie das $i$-te Element eines `bitarray` mit `bitarray[i]` - ganz so, wie Sie es sich von `array`s gewohnt sind, ansprechen können.

Wenn Sie die Zelle ausführen, indem Sie die Tasten ⬆️ + ⏎ drücken, dann stehen Ihnen die Funktionen auch in nachfolgenden Code-Zellen zur Verfügung. 

☝️ Die `assert`-Befehle am Ende der Zelle können Sie als Miniatur-Testfälle begreifen. Diese beweisen, dass die Funktionen tatsächlich das tun, was Sie zu tun vorgeben. Der Befehl `assert x` gibt einen Fehler aus, falls `x`, welches hier für eine beliebige Überprüfung steht, `false` ist, und verhält sich sonst still.

In [3]:
# TODO: Benutzen Sie die Funktionen, um die bit-Folge für "Hallo Welt" herauszufinden,
# indem Sie in diese Zelle hier den entsprechenden Code schreiben und mit ⬆️ + ⏎ ausführen.

print(message_to_bits("Hallo Welt"))
# Eine Besonderheit von jupyter ist, dass Sie, im Gegensatz zu gewöhnlichem python
# die Ausgabe der letzten Code-Zeile in einer Zelle in jedem Fall angezeigt bekommen.
# Auch, wenn Sie keinen print()-Befehl verwenden!
message_to_bits("Hallo Welt")

bitarray('01001000011000010110110001101100011011110010000001010111011001010110110001110100')


bitarray('01001000011000010110110001101100011011110010000001010111011001010110110001110100')

## Teilaufgabe 2.2

Nachfolgend erhalten Sie ein paar weitere Helfer-Funktionen, welche Sie nachfolgend benötigen werden. Hierbei legen wir bei unseren Nachrichten $m = m_0, m_1, m_2, ...$ fest, dass Sie wie folgt in Tabellen-Form übertragen werden (und vice-versa!).

0️⃣ Falls die Nachricht nicht genau eine Länge $|m|\mod{4} \equiv 0$ hat, so werden die verbleibenden bits mit `0` aufgefüllt. Wird die Tabelle zurück in ein `bitarray` konvertiert, können diese `0` bedenkenlos beibehalten werden.

| | | | |
| --- | --- | --- | --- |
| $m_0$ | $m_1$ | $m_2$ | $m_3$ |
| $m_4$ | $m_5$ | $m_6$ | $m_7$ |
| $m_8$ | $m_9$ | $m_{10}$ | $m_{11}$ |
| $m_{12}$ | $0$ | $0$ | $0$ |

In [176]:
import math

def empty_table(rows, columns):
    """
    Erstellt eine leere Tabelle als Array von bitarray in den
    Dimensionen rows ⨯ columns, und befüllt alle bits mit 0
    """
    table = []
    for i in range(0, rows):
        row = bitarray(columns)
        row.setall(0)
        table.append(row)
    return table

def copy_table(tible):
    """
    Erstellt eine Kopie von einer gegebenen Tabelle. Dies ist 
    beispielsweise nötig, wenn wir auf einer Tabelle Operationen
    vornehmen möchten, aber uns ein unverändertes Original be-
    halten möchten.
    """
    table = []
    for row in tible:
        table.append(bitarray([ bit for bit in row ]))
    return table

def print_table(table):
    """
    Ermöglicht uns, Tabellen einfach für einen Menschen
    zu lesen darzustellen
    """
    for row in table:
        for bit in row:
            print(bit, end=" ")
        print()
    print()
        
def get_column(table, index):
    """
    Liefert alle bits in einer Zeile einer Tabelle als bitarray zurück
    """
    column = bitarray()
    for row in table:
        column.append(row[index])
    return column

def bits_to_table(bits, columns=4):
    """
    Wandelt ein bitarray in eine Tabelle mit columns Spalten.
    Falls kein columns-Wert angegeben wird, dann wird 4 als Standart-
    Wert angenommen.
    """
    rows = math.ceil(len(bits) / columns)
    table = empty_table(rows, columns)
        
    for i, bit in enumerate(bits):
        row = int(i / columns)
        table[row][i % columns] = bit
    return table

def table_to_bits(table):
    """
    Wandelt eine Tabelle in ein einzelnes bitarray um,
    in dem es ein neues bitarray erstellt, und für jede Zeile
    jedes bit anhängt.
    """
    
    # TODO: Vervollständigen Sie die nachfolgende Funktion.
    # Verwenden Sie hierzu bits.append(). Die nachfolgenden
    # assert-Befehle sollten keine Fehler mehr ausgeben.
    bits = bitarray()
    
    for row in table:
        for bit in row:
            bits.append(bit)
            
    return bits

table = empty_table(5, 5)
assert len(table) == 5
assert len(table[2]) == 5
bits = bitarray('0100100001100101011011000110110001101111')
table = bits_to_table(bits)
assert len(table) == 10
assert bits == table_to_bits(bits_to_table(bits))
assert get_column(table,0) == bitarray('0100010101')

🙅‍♂️ Um ein Bit zu vertauschen, verwenden Sie den $\oplus$-Operanden. In `python` wird dieser mit dem Zeichen `^` beschrieben!

In [186]:
import random

def corrupt_bits(bats, victim_count):
    
    # copy to keep original bits as they are
    # since python passes by object reference
    bits = bitarray([ bit for bit in bats ])
    
    for i in range(0, victim_count):
        victim = random.randint(0, len(bits)-1)
        bits[victim] ^= 1
    return bits

def corrupt_table(tible, victim_count):
    
    # copy to keep original bits as they are
    # since python passes by object reference
    table = copy_table(tible)
    
    for i in range(0, victim_count):
        victim_row = random.randint(0, len(table)-1)
        victim_column = random.randint(0, len(table[0])-1)
        table[victim_row][victim_column] ^= 1
    return table

bits = bitarray('0100100001100101011011000110110001101111')
assert not (bits == corrupt_bits(bits, 1))

In [174]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def transmit(message, unreliable):
    bits = message_to_bits(message)
    bats = corrupt_bits(bits, unreliable)
    return bits_to_message(bats)

message = "Hello, my name is John Wayne and I'm here to blow your mind!"
interact(transmit, message=message, unreliable=(0,20));

interactive(children=(Text(value="Hello, my name is John Wayne and I'm here to blow your mind!", description='…

## Teilaufgabe 2.2

Nun dass wir unser Testgeschirr bereit haben, und testen können, ob unsere Kodierung auch tatsächlich Fehler erkennt und korrigiert, geht's an die Arbeit! Wir beginnen mit einer einfachen Implemetierung der Hamming-Kodierung, in welcher wir die Größe unserer "Kartentrick-Tabelle", wie Sie es aus dem Buch kennen, nicht optimal in Bezug auf die Länge der Kodierung wählen, sondern auf 4 Spalten fixieren. 

⚠️ Erinnern Sie sich, dass in der Mathematik das erste Element einer Nachricht mit $1$ bezeichnet wird, in `python` aber mit `0`! Wir verwenden nachfolgend Letzteres.

Eine Nachricht $m$ der Länge $n=13$, beispielsweise, wird also wie folgt dargestellt. Bemerken Sie, wie die Nachricht mit $0$ aufgefüllt wird, um die Tabelle zu vervollständigen:

| $c_0$ | $c_1$ | $c_2$ | $c_3$ | $c_{8*}$ |
| --- | --- | --- | --- | --- |
| $m_0$ | $m_1$ | $m_2$ | $m_3$ | $c_4$ |
| $m_4$ | $m_5$ | $m_6$ | $m_7$ | $c_5$ |
| $m_8$ | $m_9$ | $m_{10}$ | $m_{11}$ | $c_6$ |
| $m_{12}$ | $0$ | $0$ | $0$ | $c_7$ |

In [88]:
import math

def how_many_control_bits(bits):
    columns = 4 + 1
    rows = math.ceil(len(bits) / 4)
    return columns + rows

def how_many_message_bits(bits):
    return ( len(bits) - 5 ) * 0.8

# Wenn Sie keinen Fehler erhalten, dann stimmt Ihre Funktion vermutlich!
assert how_many_control_bits(bitarray("1")) == 6
assert how_many_control_bits(bitarray("1000")) == 6
assert how_many_control_bits(bitarray("100011")) == 7
assert how_many_control_bits(bitarray("100010001000")) == 8
assert how_many_control_bits(bitarray("1000100010001")) == 9

Zur Veranschaulichung nehmen Sie folgendes Beispiel:

| $c_0=1$ | $c_1=0$ | $c_2=1$ | $c_3=1$ | $c_{8*}=1$ |
| --- | --- | --- | --- | --- |
| $0$ | $1$ | $1$ | $1$ | $c_4=1$ |
| $1$ | $1$ | $0$ | $1$ | $c_5=1$ |
| $1$ | $0$ | $0$ | $1$ | $c_6=0$ |
| $1$ | $0$ | $0$ | $0$ | $c_7=1$ |

Übertragen wird die kodierte Nachricht $m'$ in unserem Beispiel wie folgt:

| | | | | | 
| --- | --- | --- | --- | --- |
| $m'_0=c_0=1$ | $m'_1=c_1=0$ | $m'_2=c_2=1$ | $m'_3=c_3=1$ | $m'_4=c_{8*}=1$ |
| $m'_5=m_0=0$ | $m'_6=m_1=1$ | $m'_7=m_2=1$ | $m'_8=m_3=1$ | $m'_9=c_4=1$ |
| $m'_{10}=m_4=1$ | $m'_{11}=m_5=1$ | $m'_{12}=m_6=0$ | $m'_{13}=m_7=1$ | $etc...$ |

In [112]:
def encode(table):
    encoded_rows = len(table) + 1
    encoded_columns = 5
    encoded_table = empty_table(encoded_rows, encoded_columns)
    
    for r, row in enumerate(table):
        for c, bit in enumerate(row):
            encoded_table[r+1][c] = bit
            encoded_table[r+1][4] ^= bit
    
    for row in encoded_table[1:]:
        for column in range(0, encoded_columns):
            encoded_table[0][column] ^= row[column]
            
    return encoded_table

def decode(encoded_table):
    decoded_table = []
    for row in encoded_table[1:]:
        decoded_table.append(row[:-1])
    return decoded_table
    
bits = bitarray('0111110110011000')
table = bits_to_table(bits)
assert encode(table)[0] == bitarray('10111')
assert decode(encode(table)) == table

In [179]:
def is_paris(bits):
    parity = 1
    for bit in bits:
        parity ^= bit
    return parity

def correct(tible):
    
    # copy to keep original bits as they are
    # since python passes by object reference
    table = copy_table(tible)
    
    for r, row in enumerate(table):
        if not is_paris(row):
            for c in range(0, len(row)):
                column = get_column(table, c)
                if not is_paris(column):
                    table[r][c] ^= 1
                    break
    return table

table = bits_to_table(bitarray("1011101111110111001010001"), 5)
assert table == correct(corrupt_table(table, 1))

In [193]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def test_encoding(message, unreliable):
    table = bits_to_table(message_to_bits(message), 4)
    encoded = encode(table)
    
    corrupted = corrupt_table(encoded, unreliable)
    corrected = correct(corrupted)
    
    print(f"You entered:\t\t'{message}'")
    print(f"After transmission:\t'{bits_to_message(table_to_bits(decode(corrupted)))}'")
    print(f"With correction:\t'{bits_to_message(table_to_bits(decode(corrected)))}'")
    
test_encoding("I'm John Wyne", 1)

message = "Hello, my name is John Wayne and I'm here to blow your mind!"
interact(test_encoding, message=message, unreliable=(0,5));

You entered:		'I'm John Wyne'
After transmission:	'H'm John Wyne'
With correction:	'I'm John Wyne'


interactive(children=(Text(value="Hello, my name is John Wayne and I'm here to blow your mind!", description='…